# 1917 영화 댓글 Preprocessing

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import math
import time

## Naver 영화 페이지의 '1917'영화 댓글 Crawling

In [2]:
# Chromedriver로 chrome browser 실행
path = '/Users/wolee-mac/WorkSpace/chromedriver'
browser = webdriver.Chrome( path )

# Naver 영화 페이지의 '1917'영화 페이지 이동
url = 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=187321'
browser.get( url )

# 평점에 더보기 링크 클릭
view_more = browser.find_element_by_xpath( '//*[@id="content"]/div[1]/div[4]/div[5]/div[2]/div[1]/a' )
view_more.click()

browser.switch_to_frame( browser.find_element_by_id( 'pointAfterListIframe' ) )

/Users/wolee-mac/opt/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: use driver.switch_to.frame instead
  del sys.path[0]


In [3]:
# 전체 댓글수에 따른 페이지 계산
html0 = browser.page_source
html1 = BeautifulSoup( html0, 'lxml' )
html2 = html1.find( 'div', { 'class': 'score_total' } ).find( 'strong' ).findChildren( 'em' )[ 0 ].getText()
total_comment = int( html2.replace(',', '') )
total_page = total_comment / 10

print( total_comment )
print( total_page )
print( math.trunc( total_page ) )

3096
309.6
309


In [4]:
movie_comments = []

for page in range( 0, math.trunc( total_page ) ): # 전체 페이지를 이동
    time.sleep( 1 ) #1초의 시간 delay
    html0 = browser.page_source #selenuim에서 현재 있는 browser의 페이지 소스 불러오기
    html1 = BeautifulSoup( html0, 'lxml') #bs4를 이용하여 html로 파싱하기
    html2 = html1.find( 'div', { 'class': 'ifr_area basic_ifr' } ) #댓글을 포함하고 있는 tag찾기
    review0 = html2.find( 'div', { 'class': 'score_result' } ).find_all( 'li' ) #각 댓글별로 list화
    
    for i in range( len( review0 ) ): #한페이지 내에서 모든 댓글을 수집하기 위한 반복문
        review = review0[ i ].find( 'div',{ 'class': 'score_reple' } ).find( 'span' ).text.strip() #댓글
        if review not in '관람객': # 불필요한 내용 삭제
            movie_comments.append( review + '\n' )
        
    #다음페이지로 넘어가기
    if page == 0:
        browser.find_elements_by_xpath('//*[@class = "paging"]/div/a')[10].click()
    else:
        browser.find_elements_by_xpath('//*[@class = "paging"]/div/a')[11].click()
